<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Linkedin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
We have a dataset of LinkedIn users, where each record contains details about their work history — employer,
 job position, and the start and end dates of each job.

We want to find out how many users had Microsoft as their employer, and immediately after that, they started
working at Google, with no other employers between these two positions.

In [8]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

spark=SparkSession.builder.appName("learning").getOrCreate()

linkedin_data = [
    (1, 'Microsoft', 'developer', '2020-04-13', '2021-11-01'),
    (1, 'Google', 'developer', '2021-11-01', None),
    (2, 'Google', 'manager', '2021-01-01', '2021-01-11'),
    (2, 'Microsoft', 'manager', '2021-01-11', None),
    (3, 'Microsoft', 'analyst', '2019-03-15', '2020-07-24'),
    (3, 'Amazon', 'analyst', '2020-08-01', '2020-11-01'),
    (3, 'Google', 'senior analyst', '2020-11-01', '2021-03-04'),
    (4, 'Google', 'junior developer', '2018-06-01', '2021-11-01'),
    (4, 'Google', 'senior developer', '2021-11-01', None),
    (5, 'Microsoft', 'manager', '2017-09-26', None),
    (6, 'Google', 'CEO', '2015-10-02', None)
]

# Define the schema for the LinkedIn data
linkedin_columns = ['user_id', 'employer', 'position', 'start_date', 'end_date']


df=spark.createDataFrame(linkedin_data,linkedin_columns)

df.show()

window_spec=Window.partitionBy('user_id').orderBy('start_date')

df1=df.withColumn('next_employer',lead('employer').over(window_spec))

df1.show()

df2=df1.filter("employer='Microsoft' and next_employer='Google'")

df2.show()

df3=df2.select("user_id").distinct().count()

print(df3)

+-------+---------+----------------+----------+----------+
|user_id| employer|        position|start_date|  end_date|
+-------+---------+----------------+----------+----------+
|      1|Microsoft|       developer|2020-04-13|2021-11-01|
|      1|   Google|       developer|2021-11-01|      NULL|
|      2|   Google|         manager|2021-01-01|2021-01-11|
|      2|Microsoft|         manager|2021-01-11|      NULL|
|      3|Microsoft|         analyst|2019-03-15|2020-07-24|
|      3|   Amazon|         analyst|2020-08-01|2020-11-01|
|      3|   Google|  senior analyst|2020-11-01|2021-03-04|
|      4|   Google|junior developer|2018-06-01|2021-11-01|
|      4|   Google|senior developer|2021-11-01|      NULL|
|      5|Microsoft|         manager|2017-09-26|      NULL|
|      6|   Google|             CEO|2015-10-02|      NULL|
+-------+---------+----------------+----------+----------+

+-------+---------+----------------+----------+----------+-------------+
|user_id| employer|        position|start